# Encoder Coverage for D > 1 on BD

Meant to be the counterpart to the encoder coverage theory plot in the paper. 
A lot trickier to do with BD because estimating the encoders takes a lot of time.

In [ ]:
%load_ext autoreload
%autoreload 2

from pystorm.hal import HAL
from pystorm.PyDriver import bddriver as bd
from pystorm.hal.net_builder import NetBuilder
from pystorm.hal.calibrator import Calibrator, PoolSpec

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [ ]:
def get_random_unit_vector(D):
    gaussian = np.random.randn(D)
    return gaussian / np.linalg.norm(gaussian)

def get_random_unit_vectors(D, N):
    vects = np.zeros((N, D)) # same shape as encoders
    for n in range(N):
        vects[n,:] = get_random_unit_vector(D)
    return vects

def dot_to_angle(dots):
    # 2 * a * b * cos(theta) = dot
    return np.arccos(dots) / np.pi * 180
        
def get_max_dots_between(vects1, vects2):
    # expecting NxD inputs
    assert(vects1.shape[1] == vects2.shape[1])
    
    # do dot produce, get N1xN2 angle matrix
    sim = np.dot(vects1, vects2.T)
    
    maxes1 = np.max(sim, axis=1)
    maxes2 = np.max(sim, axis=0)
    
    return maxes1, maxes2

def test_encoders(normed_encs, num_cmp_points):
    # generate test points, get dot products
    D = normed_encs.shape[1]
    test_vs = get_random_unit_vectors(D, num_cmp_points)
    enc_dots, test_v_dots = get_max_dots_between(normed_encs, test_vs)
    return test_v_dots

def get_performance(test_v_dots, pctile=.1):
    sorted_dots = np.sort(test_v_dots)
    p = sorted_dots[int(pctile * len(test_v_dots))]
    return p

In [ ]:
hal = HAL()
cal = Calibrator(hal)

D = 3
Y, X = 16, 16
N = Y * X
LY, LX = 0, 0

sanity_TPM = np.zeros((N, D))
for y in range(0, Y, 2):
    for x in range(0, X, 2):
        if y > Y // 2:
            n = y * X + x
            sanity_TPM[n, 1] = 1
sanity_TPM[0, 0] = 1
sanity_TPM[2, 0] = 1
sanity_TPM[4, 0] = 1
sanity_TPM[6, 0] = 1

#sanity_TPM[0 + 32, 2] = 1
#sanity_TPM[2 + 32, 2] = 1
#sanity_TPM[4 + 32, 2] = 1
#sanity_TPM[6 + 32, 2] = 1

## this one had something weird
#sanity_TPM = np.zeros((N, D))
#for y in range(0, Y, 2):
#    for x in range(0, X, 2):
#        n = y * X + x
#        if (y // 2) % 2 == 0: # d+
#            if x > X // 2: # d0
#                sanity_TPM[n, 0] = 1
#            else: # d1
#                sanity_TPM[n, 1] = 1
#        else: # d-
#            if x > X // 2: # d0
#                sanity_TPM[n, 0] = -1
#            else: # d1
#                sanity_TPM[n, 1] = -1

#sanity_TPM = np.zeros((N, D))
#for y in range(0, Y, 2):
#    for x in range(0, X, 2):
#        n = y * X + x
#        if (y // 2) % 3 == 0: # d0
#            sanity_TPM[n, 0] = 1
#        if (y // 2) % 3 == 1: # d0
#            sanity_TPM[n, 1] = 1
#        if (y // 2) % 3 == 2: # d0
#            sanity_TPM[n, 2] = 1
#

sanity_TPM_yx = sanity_TPM.reshape((Y, X, D))
for d in range(D):
    plt.figure()
    plt.imshow(sanity_TPM_yx[:, :, d])

#base_ps = PoolSpec(YX=(Y,X), loc_yx=(LY, LX), D=D, TPM=sanity_TPM)


# ignore bad taps--yield doesn't matter here
SY, SX = NetBuilder.to_synspace(Y, X)                                                   
syn_yx_tap_matrix = NetBuilder.create_default_yx_taps(SY, SX, D, bad_syn=None)
                                                                                        
# reshape to NxD shape and make even                                                    
default_TPM = NetBuilder.syn_taps_to_nrn_taps(syn_yx_tap_matrix)  
NetBuilder.make_taps_even(default_TPM)

base_ps = PoolSpec(YX=(Y,X), loc_yx=(LY, LX), D=D, TPM=default_TPM)

#base_ps = PoolSpec(YX=(Y,X), loc_yx=(LY, LX), D=D)

bad_syns, _ = cal.get_bad_syns()
print(bad_syns.shape)
pool_bad_syns = Calibrator.crop_calibration_to_pool(bad_syns, base_ps)  
print(pool_bad_syns.shape)

plt.figure()
plt.imshow(~pool_bad_syns)
plt.title('good synapses')

if D == 3:
    DAC_vals = {
            'DAC_SOMA_REF': 1024,
            'DAC_DIFF_G': 1024,
            'DAC_DIFF_R': 100}
else:
    DAC_vals = {}

# yield doesn't actually come into play for the metric, 
# but we're going to be hard on ourselves and only include neurons that fire
# not just 'good', for starters

In [ ]:
def optimize_net(base_ps):
    return cal.optimize_yield(base_ps, dacs=DAC_vals, 
                           bias_twiddle_policy='center', offset_source='calibration_db', validate=False, 
                           get_encs_kwargs={'solver':'scipy_opt'})


In [ ]:
#base_ps.TPM = opt_ps.TPM

In [ ]:
NUM_TRIALS = 1
pss = []
encs = []
offs = []
std_encs = []
std_offs = []
dbgs = []
for i in range(NUM_TRIALS):
    print("RUNNING TRIAL", i)
    print("==================")
    
    #if i > 0:
    #    base_ps.TPM = pss[0].TPM
    
    opt_ps, dac_vals, opt_encs, opt_offsets, std_opt_encs, std_opt_offsets, dbg = optimize_net(base_ps)
    encs.append(opt_encs)
    offs.append(opt_offsets)
    std_encs.append(std_opt_encs)
    std_offs.append(std_opt_offsets)
    pss.append(opt_ps)
    dbgs.append(dbg)
    
    # save progress
    import pickle
    pickle.dump((pss, encs, offs, std_encs, std_offs), open('save_encs_and_offs3.pck', 'wb'))

In [ ]:
def analyze_net(opt_ps, opt_encs, opt_offsets, std_encs, std_offsets, scatter=False):
    intercepts, good = cal.get_intercepts(opt_encs, opt_offsets)
    plt.figure()
    gains = cal.get_gains(opt_encs)
    print(len(gains))
    big_gain_mask = gains > 50
    print(len(big_gain_mask))
    big_gains = gains[big_gain_mask]
    print(len(big_gains), 'substantial gains')

    plt.figure()
    plt.hist(big_gains, bins=40)

    big_encs = opt_encs[big_gain_mask, :]
    normed_big_encs = (big_encs.T / big_gains).T
    
    big_std_encs = std_encs[big_gain_mask, :]
    normed_std_encs = (big_std_encs.T / big_gains).T

    if scatter:
        
        def make_one_scatter(ax, encs, d0, d1, std_encs=None):
            m = np.max(np.abs(encs[~np.isnan(encs)]))
            if m > 500:
                m = 500 
            ax.axis([-m, m, -m, m])
            ax.scatter(encs[:, d0], encs[:, d1], s=4)
            if std_encs is not None:
                from matplotlib.patches import Ellipse
                for i in range(encs.shape[0]):
                    r0 = std_encs[i, d0] * 2 # 2-sigma ~ 95%
                    r1 = std_encs[i, d1] * 2
                    ell = Ellipse((encs[i, d0], encs[i, d1]), r0, r1)
                    ax.add_artist(ell)
                    ell.set_alpha(.2)
                    ell.set_facecolor('orange')
            
        if D == 2:
            fig, ax = plt.subplots(1, 2, figsize=(2*7, 7))
            make_one_scatter(ax[0], big_encs, 0, 1, std_encs=std_encs)
            make_one_scatter(ax[1], normed_big_encs, 0, 1, std_encs=normed_std_encs)
        if D == 3:
            fig, ax = plt.subplots(3, 2, figsize=(2*7, 3*7))
            make_one_scatter(ax[0, 0], big_encs, 0, 1, std_encs)
            make_one_scatter(ax[1, 0], big_encs, 1, 2, std_encs)
            make_one_scatter(ax[2, 0], big_encs, 2, 0, std_encs)
            make_one_scatter(ax[0, 1], normed_big_encs, 0, 1, normed_std_encs)
            make_one_scatter(ax[1, 1], normed_big_encs, 1, 2, normed_std_encs)
            make_one_scatter(ax[2, 1], normed_big_encs, 2, 0, normed_std_encs)
    
    plt.figure(figsize=(5,5))
    all_tap_ax = plt.gca()
    
    fig, ax = plt.subplots(1, D+1, figsize=(3*D, 3), gridspec_kw={'width_ratios':[20]*D + [1]})
    for d in range(D):
        thr_encs = opt_encs[:, d].copy()
        thr_encs[np.isnan(thr_encs)] = 0
        #im = ax[d].imshow(thr_encs.reshape(Y, X), vmin=-800, vmax=800)
        
        log_encs = thr_encs.copy()
        log_encs[log_encs > 0] = np.log(log_encs[log_encs > 0] + 1)
        log_encs[log_encs < 0] = -np.log(-log_encs[log_encs < 0] + 1)
        
        im = ax[d].imshow(log_encs.reshape(Y, X), vmin=-np.log(800), vmax=np.log(800))
        
        if d == D-1:
            plt.colorbar(im, cax=ax[d+1])
        
        pos_taps = []
        neg_taps = []
        for x in range(opt_ps.X):
            for y in range(opt_ps.Y):
                n = y * X + x
                xsyn = x
                if (y // 2) % 2 == 1:
                    xsyn += 1
                if opt_ps.TPM[n, d] == 1:
                    pos_taps.append([y, xsyn])
                if opt_ps.TPM[n, d] == -1:
                    neg_taps.append([y, xsyn])
        pos_taps = np.array(pos_taps)
        neg_taps = np.array(neg_taps)
        if len(pos_taps) > 0:
            ax[d].scatter(pos_taps[:, 1], pos_taps[:, 0], marker='+', c='r')
        if len(neg_taps) > 0:
            ax[d].scatter(neg_taps[:, 1], neg_taps[:, 0], marker='+', c='cyan')
            
        dim_colors = ['r', 'g', 'b', 'cyan', 'magenta', 'yellow']
        all_tap_ax.scatter(pos_taps[:, 1], pos_taps[:, 0], marker='x', c=dim_colors[d])
        all_tap_ax.scatter(neg_taps[:, 1], neg_taps[:, 0], marker='o', c=dim_colors[d])
    plt.tight_layout(w_pad=.05, h_pad=.05)
            
        
    num_pts = 1000
    dots = test_encoders(normed_big_encs, num_pts)

    plt.figure()
    plt.plot(np.sort(dots))
    plt.axvline(.1 * num_pts)

    return get_performance(dots)

In [ ]:
import pickle
pss, encs, offs, std_encs, std_offs = pickle.load(open('save_encs_and_offs3.pck', 'rb'))
perfs = []
for i, ps, enc, off, std_enc, std_off in zip(range(len(encs)), pss, encs, offs, std_encs, std_offs):
    print("ANALYZING TRIAL", i)
    print("==================")
    perfs.append(analyze_net(ps, enc, off, std_enc, std_off, scatter=True))
print(perfs)

In [ ]:
spikes = dbgs[0]['spikes']
sample_pts = dbgs[0]['sample_pts']
print(spikes.shape)
print(sample_pts.shape)
plt.figure()
gains = Calibrator.get_gains(encs[0])
nrn = 32
print('gain', gains[nrn])
plt.tricontourf(sample_pts[:, 0], sample_pts[:, 1], spikes[:, nrn])
plt.colorbar()
print(encs[0][nrn, :])
plt.figure()
plt.plot(spikes[:, nrn])

In [ ]:
plt.figure()
plt.hist(perfs, bins=20)
print(max(perfs))

In [ ]:
# neurons get more juice regardless of whether dim 0 or dim 1 is driven
# both SGs target the same place in TAT?
# SG0 shouldn't target anything?

In [ ]:
np.sum(np.abs(pss[0].TPM))

In [ ]:
ints, gains = Calibrator.get_intercepts(encs[0], offs[0])
plt.figure()
plt.hist(ints[~np.isnan(ints)], range=(-4, 0), bins=20)

- improve algorithm to be smarter when taps are empty:
    - or just don't ignore 'bad taps' -- easy, maybe do this for now
    - should actually spend some time with the basic algorithm. Doesn't seem to do a good job keeping taps/dim 
    relatively constant
    - should perhaps bootstrap the 'best' taps results from the simulation to the chip
        - need to resurrect notebook. would be good to do that before Ben gets his hands on it
- need to get a handle on the encoder estimation error thing, wrt presenting results in paper
    - bootstrapping to get the best performance's variance probably isn't actually the right thing to do
    - bootstrapping the error of a GIVEN encoder might be a better thing to do
    - since the statistic is on the closest encoder, this tells that the closet encoder could be up to this much 
    - farther away
    
### Game Plan 
    
1. finish bootstrap
2. see if 3^D -> 2^D points tanks estimation quality
3. update CDF to use 2-sigma worst-case
    - figure out how many samples are needed for reasonably distance from nominal performance
4. go to unconstrained anchor encoders
    - add in pre-optimization simulation
5. off to the races!?